In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import f1_score
from sklearn.externals import joblib
from pyod.utils.data import generate_data
from pyod.utils.data import get_color_codes
from pyod.utils.data import evaluate_print
from pyod.models.ocsvm import OCSVM

from __future__ import division
from __future__ import print_function

import os
import sys


sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager

In [2]:
inlier_train = pd.read_csv('inlier.csv',header=None)
outlier_train =  pd.read_csv('outlier.csv',header=None)
#test =  pd.read_csv('test.csv',header=None)

In [3]:
y_inlier=np.zeros( (inlier_train.shape[0],1), dtype=np.int16 )
y_outlier=np.ones( (outlier_train.shape[0],1), dtype=np.int16 )

In [4]:
X= pd.concat([inlier_train,outlier_train], axis=0)

In [5]:
y=np.append(y_inlier,y_outlier)

In [6]:
# apply standard scaler to output from resnet50
ss = StandardScaler()
ss.fit(X)
x_train = ss.transform(X)
#test_x130 = ss.transform(test)#####小写der

In [7]:
outlier_per=outlier_train.shape[0]/(outlier_train.shape[0]+inlier_train.shape[0])

In [8]:
X_train, X_test, y_train, y_test = train_test_split( x_train, y, test_size=0.3, random_state=42)

In [10]:
outliers_fraction = outlier_per  # percentage of outliers
clf_name = 'OCSVMt'
clf = OCSVM(contamination=outliers_fraction)
clf.fit(X_train)
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


On Training Data:
OCSVMt ROC:0.9059, precision @ rank n:0.4875

On Test Data:
OCSVMt ROC:0.9122, precision @ rank n:0.5746


In [11]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_test_pred)

array([[1373,   58],
       [  57,   77]], dtype=int64)

In [12]:
full_test=pd.read_csv('test_full.csv',header=None)
submission=pd.DataFrame()
submission['ID']=full_test.iloc[:,-1]

In [13]:
X_testfull=full_test.drop(columns=2048)
#X

In [14]:
X_testfull=ss.transform(X_testfull)

In [16]:
y_test_pred = clf.predict(X_testfull) 
y_test_scores =clf.decision_function(X_testfull)
submission['Result']=y_test_pred
submission.to_csv('submission.csv',index=False)

In [17]:
outlier_per

0.07920981971614884